In [13]:
import os
import cv2
import numpy as np

# Paths
input_base  = r"C:\Users\Hardika\Desktop\NNpro1_7species\Original_nnpro_7"
output_base = r"C:\Users\Hardika\Desktop\NNpro1_7species\thresh_contour_7sp"
os.makedirs(output_base, exist_ok=True)

# Augment settings
rotation_angles   = [0, 90, 180]
contrast_factors  = [1.0, 1.3]

for species in os.listdir(input_base):
    in_dir  = os.path.join(input_base, species)
    out_dir = os.path.join(output_base, species)
    if not os.path.isdir(in_dir): continue
    os.makedirs(out_dir, exist_ok=True)

    for fname in os.listdir(in_dir):
        # 1) load & resize
        img = cv2.imread(os.path.join(in_dir, fname), cv2.IMREAD_GRAYSCALE)
        if img is None: continue
        img = cv2.resize(img, (256,256))

        # 2) your adaptive threshold (leaf = 0, bg+veins = 255)
        th = cv2.adaptiveThreshold(
            img, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY,
            11, 2
        )

        # 3) invert for contour-detection (leaf=255, bg+veins=0)
        inv = cv2.bitwise_not(th)

        # 4) find & fill only large contours (the leaf)
        cnts, _ = cv2.findContours(inv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        leaf_mask = np.zeros_like(th)
        for c in cnts:
            if cv2.contourArea(c) > 500:               # tune this threshold to your leaves
                cv2.drawContours(leaf_mask,[c],-1,255,-1)

        # 5) invert leaf_mask → background=255, leaf=0
        leaf_mask_inv = cv2.bitwise_not(leaf_mask)

        # 6) combine with original threshold to keep veins but remove noise
        #    background stays white (255), leaf body stays black (0), veins stay white (255)
        clean = cv2.bitwise_or(th, leaf_mask_inv)

        # 7) augment: rotations + contrast
        for angle in rotation_angles:
            if angle == 90:   aug = cv2.rotate(clean, cv2.ROTATE_90_CLOCKWISE)
            elif angle == 180: aug = cv2.rotate(clean, cv2.ROTATE_180)
            else:              aug = clean.copy()

            for cf in contrast_factors:
                out = cv2.convertScaleAbs(aug, alpha=cf, beta=0)
                name, _ = os.path.splitext(fname)
                save_path = os.path.join(out_dir, f"{name}_r{angle}_c{str(cf).replace('.','')}.jpg")
                cv2.imwrite(save_path, out)

print(" Done: binary masks with white bg, black leaf, white veins, noise removed.") 


 Done: binary masks with white bg, black leaf, white veins, noise removed.
